# Domain Generation Algorithm (DGA) Detection

## Table of Contents
* Introduction
* Data Importing
* Data Preprocessing
* Training and Evaluation
* Inference
* Conclusion

## Introduction
[Domain Generation Algorithms](https://en.wikipedia.org/wiki/Domain_generation_algorithm) (DGAs) are used to generate domain names that can be used by the malware to communicate with the command and control servers. IP addresses and static domain names can be easily blocked, and a DGA provides an easy method to generate a large number of domain names and rotate through them to circumvent traditional block lists. We will use a type of recurrent neural network called the [Gated Recurrent Unit](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21) (GRU) for this example. This implementation enables users to train their models with up-to-date domain names representative of both benign and DGA generated strings. This capability could also be used in production. This notebook provides a view into the data science workflow to create a DGA detection implementation.

In [1]:
import os
import cudf
import torch
import requests
import logging
import numpy as np
from datetime import datetime
from sklearn.metrics import accuracy_score, average_precision_score
from dga_detector import DGADetector
from dataloader import DataLoader
from dga_dataset import DGADataset
from cuml.model_selection import train_test_split

#### Enable console logging

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler()
    ]
)

#### Download Input Dataset from S3

In [3]:
INPUT_CSV = "../datasets/benign_and_dga_domains.csv"

In [4]:
# Read Benign and DGA dataset
if not os.path.exists(INPUT_CSV):
    r = requests.get(DATA_BASE_URL + INPUT_CSV)
    open(INPUT_CSV, 'wb').write(r.content)

#### Load Input Dataset to GPU Dataframe

In [5]:
gdf = cudf.read_csv(INPUT_CSV)

In [6]:
train_data = gdf['domain']
labels = gdf['type']

Because we have only benign and DGA (malicious) categoriesm, the number of domain types need to be set to 2 (`N_DOMAIN_TYPE=2`). Vocabulary size(`CHAR_VOCAB`) is set to 128 ASCII characters. The values below set for `HIDDEN_SIZE`, `N_LAYERS` of the network, and the `LR` (Learning Rate) give an optimum balance for the network size and performance. They might need be set via experiments when working with other datasets.

In [7]:
LR = 0.001
N_LAYERS = 3
CHAR_VOCAB = 128
HIDDEN_SIZE = 100
N_DOMAIN_TYPE = 2

#### Instantiate DGA detector
Now that the data is ready, the datasets are created, and we've set the parameters for the model, we can use the DGA detector method to create and train the model.

In [8]:
dd = DGADetector(lr=LR)
dd.init_model(n_layers=N_LAYERS, char_vocab=CHAR_VOCAB, hidden_size=HIDDEN_SIZE, n_domain_type=N_DOMAIN_TYPE)

2023-05-02 21:06:44,204 [INFO] Found GPU's now setting up cuda for the model


In [9]:
EPOCHS = 25
TRAIN_SIZE = 0.7
BATCH_SIZE = 10000
MODELS_DIR = 'models'

### Training and Evaluation
Now we train and evaluate the model.

In [10]:
%%time
dd.train_model(train_data, labels, batch_size=BATCH_SIZE, epochs=EPOCHS, train_size=0.7)

2023-05-02 21:06:46,283 [INFO] Initiating model training ...
2023-05-02 21:06:46,284 [INFO] Truncate domains to width: 100
/opt/conda/envs/mor_exp/lib/python3.8/site-packages/cudf/core/column/string.py:3563: FutureWarning: The expand parameter is deprecated and will be removed in a future version. Set expand=False to match future behavior.
  warnings.warn(
Epoch:   0%|          | 0/25 [00:00<?, ?it/s]2023-05-02 21:06:49,969 [INFO] [100000/1433083 (7%)]	Loss: 6938.24
2023-05-02 21:06:52,000 [INFO] [200000/1433083 (14%)]	Loss: 5141.84
2023-05-02 21:06:53,884 [INFO] [300000/1433083 (21%)]	Loss: 3975.97
2023-05-02 21:06:55,626 [INFO] [400000/1433083 (28%)]	Loss: 3446.93
2023-05-02 21:06:57,307 [INFO] [500000/1433083 (35%)]	Loss: 3107.64
2023-05-02 21:06:58,829 [INFO] [600000/1433083 (42%)]	Loss: 2693.71
2023-05-02 21:07:00,312 [INFO] [700000/1433083 (49%)]	Loss: 2367.30
2023-05-02 21:07:01,940 [INFO] [800000/1433083 (56%)]	Loss: 2107.43
2023-05-02 21:07:03,584 [INFO] [900000/1433083 (63%)]

CPU times: user 21min 21s, sys: 19.2 s, total: 21min 40s
Wall time: 10min 55s


### Save Model
Save pretrained model to a given output location.

In [11]:
if not os.path.exists(MODELS_DIR):
    print("Creating directory '{}'".format(MODELS_DIR))
    os.makedirs(MODELS_DIR)

now = datetime.now()
model_filename = "rnn_classifier_{}.bin".format(now.strftime("%Y-%m-%d_%H_%M_%S"))
model_filepath = os.path.join(MODELS_DIR, model_filename)
dd.save_checkpoint(model_filepath)

2023-05-02 21:17:41,976 [INFO] Pretrained model checkpoint saved to location: 'models/rnn_classifier_2023-05-02_21_17_41.bin'


Creating directory 'models'


### Inference

Using the model generated above, we now score the test dataset against the model to determine if the domain is likely generated by a DGA or benign.

In [12]:
dga_detector = DGADetector()
dga_detector.load_checkpoint(model_filepath)

domain_train, domain_test, type_train, type_test = train_test_split(gdf, "type", train_size=0.7)
test_df = cudf.DataFrame()
test_df["type"] = type_test.reset_index(drop=True)
test_df["domain"] = domain_test.reset_index(drop=True)

test_dataset = DGADataset(test_df, 100)
test_dataloader = DataLoader(test_dataset, batchsize=BATCH_SIZE)

pred_results = []
true_results = []
for chunk in test_dataloader.get_chunks():
    pred_results.append(list(dga_detector.predict(chunk['domain']).values_host))
    true_results.append(list(chunk['type'].values_host))
pred_results = np.concatenate(pred_results)
true_results = np.concatenate(true_results)
accuracy_score_result = accuracy_score(pred_results, true_results)

print('Model accuracy: %s'%(accuracy_score_result))

2023-05-02 21:17:42,013 [INFO] Found GPU's now setting up cuda for the model


Model accuracy: 0.9924810193770871


In [13]:
average_precision = average_precision_score(true_results, pred_results)

print('Average precision score: {0:0.3f}'.format(average_precision))

Average precision score: 0.980


## Conclusion

This DGA detection implementation enables users to train their models for detection and also use existing models. This capability could also be used in conjunction with log parsing efforts if the logs contain domain names. Data is kept in GPU memory, removing unnecessary copy/converts and providing a 4X speed advantage over CPU only implementations. This is esepcially true with large batch sizes.